<div style="background-color: #ECF0F1; padding: 25px; text-align: center; border-radius: 15px; border: 1px solid #BDC3C7; margin-bottom: 20px;">
    <h1 style="color: #2C3E50; font-weight: bold; font-family: sans-serif; margin-bottom: 10px;">HuatuoGPT HashGuard SOP</h1>
    <h3 style="color: #16A085; font-style: italic; font-family: serif; margin-top: 0;">Secure. Verify. Trust HuatuoGPT.</h3>
</div>

### 📋 專案資訊
**執行環境**：Google Colab (T4 GPU Runtime)  
**目的**：確保下載到 Colab 的模型檔案未經中間人竄改，符合資安供應鏈安全標準。  
**驗證標準**：SHA-256 Checksum

In [ ]:
# 第一步：安裝必要套件與定義參數
# 為了進行鑑識，我們將模型下載到顯性的本地資料夾

!pip install -q huggingface_hub

from huggingface_hub import snapshot_download
import os

# 設定模型 ID (可依需求改為量化版，如 "FreedomIntelligence/HuatuoGPT-7b-8bit")
MODEL_ID = "FreedomIntelligence/HuatuoGPT-7b"
LOCAL_DIR = "./huatuo_safe_box"

print(f"⬇️ 正在下載模型 {MODEL_ID} 到 {LOCAL_DIR} ... (這可能需要幾分鐘)")

# 使用 snapshot_download 將檔案實體下載
try:
    snapshot_download(repo_id=MODEL_ID, local_dir=LOCAL_DIR, local_dir_use_symlinks=False)
    print("✅ 下載完成。")
except Exception as e:
    print(f"❌ 下載失敗: {e}")

In [ ]:
# 第二步：定義雜湊計算與查詢函式

from huggingface_hub import get_hf_file_metadata, hf_hub_url
import hashlib

def get_remote_sha256(repo_id, filename):
    """
    從 Hugging Face 伺服器查詢檔案的原始 SHA256 值 (黃金標準)
    """
    try:
        url = hf_hub_url(repo_id=repo_id, filename=filename)
        meta = get_hf_file_metadata(url)
        # LFS (Large File Storage) 檔案會有 sha256 欄位，小檔案可能是 blob_id
        if meta.lfs:
            return meta.lfs.get("sha256")
        else:
            # 非 LFS 檔案通常較小，風險較低，但嚴格來說也該驗證 blob sha
            return None
    except Exception as e:
        print(f"   [警告] 無法取得遠端 Metadata: {filename}")
        return None

def calculate_local_sha256(file_path):
    """
    計算本地檔案的 SHA256 (分塊讀取以避免 Colab RAM 溢出)
    """
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        # 每次讀取 8MB
        for byte_block in iter(lambda: f.read(8192*1024), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

In [ ]:
# 第三步：執行自動化比對稽核

# 篩選出需要檢查的關鍵檔案 (權重檔與設定檔)
files_to_check = [
    f for f in os.listdir(LOCAL_DIR)
    if f.endswith(".bin") or f.endswith(".safetensors") or f.endswith(".json") or f.endswith(".model")
]

print("🔍 開始進行供應鏈安全驗證 (SHA-256 Checksum)...\n")
print(f"{'檔名':<40} | {'狀態':<10} | {'說明'}")
print("-"*80)

pass_count = 0
fail_count = 0
skip_count = 0

for filename in files_to_check:
    file_path = os.path.join(LOCAL_DIR, filename)

    # 1. 取得遠端黃金標準
    remote_hash = get_remote_sha256(MODEL_ID, filename)

    if remote_hash:
        # 2. 計算本地雜湊
        local_hash = calculate_local_sha256(file_path)

        # 3. 比對
        if local_hash == remote_hash:
            print(f"{filename[:38]:<40} | ✅ PASS     | 雜湊一致")
            pass_count += 1
        else:
            print(f"{filename[:38]:<40} | ❌ FAIL     | 雜湊不符！(可能被竄改)")
            print(f"   >> 預期: {remote_hash}")
            print(f"   >> 實際: {local_hash}")
            fail_count += 1
    else:
        print(f"{filename[:38]:<40} | ⚠️ SKIP     | 非 LFS 檔案或無遠端雜湊")
        skip_count += 1

print("-"*80)
print(f"\n🏁 驗證結束總結：\n   通過: {pass_count}\n   失敗: {fail_count}\n   跳過: {skip_count}")

if fail_count > 0:
    print("\n🔴 警告：偵測到檔案雜湊不符，請勿使用此模型進行任何敏感操作！")
else:
    print("\n🟢 安全：關鍵權重檔驗證通過，符合供應鏈安全標準。")